In [19]:
import sys
print(sys.executable)
!{sys.executable} -m pip install pandas

/spack/conda/envs/ood-jupyterlab-4.2/bin/python
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 55.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 54.5 MB/s eta 0:00:00:00:01


In [21]:
import json
import requests
import time
# import pandas

In [6]:
with open('data/combined_esg_data_complete.json', 'r') as file:
    esg_data = json.load(file)

In [22]:
def generate_esg_summary(company, esg_info):
    prompt = f"""Provide a ESG summary for {company}. Include what they have achieved uptil now, 
    their future plans and the timeline to fulfill these plans.The esg data is as follows:\n\n"""
    prompt += f"Environment: {esg_info['e']}\n\n"
    prompt += f"Social: {esg_info['s']}\n\n"
    prompt += f"Governance: {esg_info['g']}\n\n"

    # Ollama API endpoint
    url = "http://127.0.0.1:11434/api/generate"

    # Prepare the payload
    payload = {
        "model": "llama3.2:1b",
        "prompt": prompt,
    }

    try:
        # Send a POST request to the Ollama API
        response = requests.post(url, json=payload, stream=True)

        # Check if the request was successful
        if response.status_code == 200:
            # Collect the generated text
            generated_text = ""
            for line in response.iter_lines():
                if line:
                    data = json.loads(line.decode('utf-8'))
                    generated_text += data.get('response', '')
            return generated_text.strip()
        else:
            print(f"Error {response.status_code}: {response.text}")
            return ""
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return ""


In [23]:
# esg_summaries = {}
# for company, esg_info in esg_data.items():
#     summary = generate_esg_summary(company, esg_info)
#     esg_summaries[company] = summary
#     print(f"ESG Summary for {company}:\n{summary}\n")

esg_summaries = {}
summary_data = []

for company, esg_info in esg_data.items():
    start_time = time.time()
    summary = generate_esg_summary(company, esg_info)
    end_time = time.time()
    
    # Calculate time taken and length of summary
    time_taken = end_time - start_time
    summary_length = len(summary)
    
    # Store results
    esg_summaries[company] = summary
    summary_data.append({
        "Company": company,
        "Time Taken (s)": time_taken,
        "Summary Length": summary_length
        # "Summary": summary
    })
    print(f"ESG Summary for {company}:\n{summary}\n")

# Convert summary data to a DataFrame and display
# df_summary = pd.DataFrame(summary_data)
# print(df_summary)
summary_data

ESG Summary for company_1:
This is a comprehensive report from Salesforce on its stakeholder impact for FY24. Here are the key points:

**Diversity, Equity, and Inclusion**

* Salesforce promotes equal opportunity and economic empowerment for women and underrepresented groups.
* The company defines diversity as those who identify as Black, Indigenous, or Latinx.

**Governance**

* Salesforce has a strong governance structure, with an independent Board of Directors, Audit and Finance Committees, Compensation Committee, and Nominating and Corporate Governance Committee.
* The company is committed to transparency, accountability, and compliance with laws and regulations.

**Technology**

* Salesforce emphasizes the importance of technology in increasing efficiency and transparency in supply chains and healthcare.
* The company has also made efforts to amplify the impact of social sector technology through partnerships and investments.

**Digital Health**

* Salesforce has expanded access 

[{'Company': 'company_1',
  'Time Taken (s)': 5.031407356262207,
  'Summary Length': 2223},
 {'Company': 'company_2',
  'Time Taken (s)': 2.0248351097106934,
  'Summary Length': 1792},
 {'Company': 'company_3',
  'Time Taken (s)': 2.813765525817871,
  'Summary Length': 2619},
 {'Company': 'company_4',
  'Time Taken (s)': 3.0344090461730957,
  'Summary Length': 2616},
 {'Company': 'company_5',
  'Time Taken (s)': 2.3377537727355957,
  'Summary Length': 1967},
 {'Company': 'company_6',
  'Time Taken (s)': 2.761021137237549,
  'Summary Length': 2616},
 {'Company': 'company_7',
  'Time Taken (s)': 2.8070645332336426,
  'Summary Length': 2417},
 {'Company': 'company_8',
  'Time Taken (s)': 2.446553945541382,
  'Summary Length': 2185},
 {'Company': 'company_9',
  'Time Taken (s)': 2.0349488258361816,
  'Summary Length': 1651},
 {'Company': 'company_10',
  'Time Taken (s)': 2.2819926738739014,
  'Summary Length': 2089},
 {'Company': 'company_11',
  'Time Taken (s)': 2.0153586864471436,
  'Sum

In [25]:
# Optional: Save the summaries to a file
with open('esg_summaries.json', 'w') as file:
    json.dump(esg_summaries, file, indent=4)

In [ ]:
def callLLM(prompt):
    url = "http://127.0.0.1:11434/api/generate"  # Adjusted endpoint
    payload = {
        "model": "llama3.2:1b",  # Ensure model name is correct
        "prompt": prompt,
        "stream": False
    }
    text = ""
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()  # Raises an error for HTTP codes 4xx/5xx
        print("Response Recived")
        text = json.loads(response.text)['response']
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")

    return text

In [26]:
import json
import requests
import time

# Define function to generate a summary using the Ollama API for a specific aspect
def generate_summary(prompt):
    url = "http://127.0.0.1:11434/api/generate" 
    payload = {
        "model": "llama3.2:1b",
        "prompt": prompt,
    }
    
    try:
        response = requests.post(url, json=payload, stream=True)
        if response.status_code == 200:
            generated_text = ""
            for line in response.iter_lines():
                if line:
                    data = json.loads(line.decode('utf-8'))
                    generated_text += data.get('response', '')
            return generated_text.strip()
        else:
            print(f"Error {response.status_code}: {response.text}")
            return ""
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return ""

# Define each agent as a function that generates a summary for its specific component
def environment_agent(company, environment_text):
    prompt = f"Provide a concise environmental summary for {company} based on the following information:\n\n{environment_text}\n\n"
    return generate_summary(prompt)

def social_agent(company, social_text):
    prompt = f"Provide a concise social summary for {company} based on the following information:\n\n{social_text}\n\n"
    return generate_summary(prompt)

def governance_agent(company, governance_text):
    prompt = f"Provide a concise governance summary for {company} based on the following information:\n\n{governance_text}\n\n"
    return generate_summary(prompt)

# Generate combined summaries for each company and measure time
esg_summaries = []
for company, esg_info in esg_data.items():
    start_time = time.time()
    
    # Each agent generates its part of the summary
    environment_summary = environment_agent(company, esg_info['e'])
    social_summary = social_agent(company, esg_info['s'])
    governance_summary = governance_agent(company, esg_info['g'])
    
    # Combine the summaries
    combined_summary = f"{environment_summary}\n\n{social_summary}\n\n{governance_summary}"
    time_taken = time.time() - start_time
    summary_length = len(combined_summary)
    
    # Store the combined summary and performance metrics
    esg_summaries.append({
        "Company": company,
        "Environment Summary": environment_summary,
        "Social Summary": social_summary,
        "Governance Summary": governance_summary,
        "Combined Summary": combined_summary,
        "Time Taken (s)": time_taken,
        "Summary Length": summary_length
    })

    print(f"ESG Summary for {company}:\n{combined_summary}\n")

# Convert to DataFrame for viewing and saving
# df_summary = pd.DataFrame(esg_summaries)
# print(df_summary)


ESG Summary for company_1:
The text provides a comprehensive list of definitions, indicators, and calculations related to diversity and inclusion in the workplace. Here's a breakdown of the key points:

**Definitions:**

1. **URGs (Underrepresented Groups):** Employees who identify as Black, Asian, Indigenous, Latina/o/x, Indian, or LGBTQ+.
2. **URMs (Underrepresented Minorities):** Employees who identify as Latinx/o/x or Non-binary/Non-Conforming.
3. **VP+:** The highest level of reporting in the GRI report, which includes top executives.

**Calculations:**

1. **URM representation:** Calculated by dividing the number of U.S. employees identified under at least one URG by the total number of U.S. employees.
2. **Progress against baseline:** Calculated by dividing current URM leadership percentage by the base year URM leadership percentage.

**Indicators:**

1. **Diversity and inclusion metrics for employees, including URGs and URMs:** Includes indicators such as diversity representati

In [27]:
with open('multi_agent_esg_summaries.json', 'w') as file:
    json.dump(esg_summaries, file, indent=4)